In [1]:
## 로그인

from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import pandas as pd
import time
import random
import os
driver = webdriver.Chrome('chromedriver.exe')
soup = bs(driver.page_source, 'html.parser')
driver.implicitly_wait(3)

insta_ID=input("ID를 입력해주세요 : ")
insta_PW=input("PW를 입력해주세요 : ")

driver.get('http://www.hongik.ac.kr/login.do?Refer=https://cn.hongik.ac.kr/stud/')

driver.find_element_by_name('USER_ID').send_keys(insta_ID)
driver.find_element_by_name('PASSWD').send_keys(insta_PW)

driver.find_element_by_css_selector('button').click()

## 팝업종료, 세부 접속

In [2]:
try:
    driver.find_element_by_name('no_show').click()
except:
    print("no pop-up")
driver.get('https://cn.hongik.ac.kr/stud/C/02000/02020.jsp')

## 카테고리 접근 경로

In [3]:
arr = [[4, 7, 16, 0], [4, 8, 17, 0], [5, 7, 19, 0], [5, 7, 20, 30], [5, 7, 21, 30],
       [5, 7, 22, 30], [5, 7, 23, 30], [5, 7, 24, 30], [5, 8, 19, 25], [5, 9, 19, 28],
       [5, 9, 20, 28], [5, 9, 21, 28], [5, 9, 22, 28], [5, 10, 19, 29], [5, 10, 20, 29],
       [5, 10, 21, 29], [5, 10, 22, 29], [5, 10, 23, 29], [5, 11, 19, 0], [5, 11, 20, 35],
       [5, 11, 21, 35], [5, 11, 22, 35], [5, 11, 23, 35], [5, 11, 24, 35], [5, 11, 25, 35],
       [5, 11, 26, 35], [5, 11, 27, 35], [5, 11, 28, 35], [5, 11, 29, 35], [5, 12, 19, 26],
       [5, 13, 19, 0], [5, 13, 20, 26], [5, 14, 19, 25], [5, 15, 19, 25], [5, 16, 19, 31],
       [5, 16, 20, 31], [5, 16, 21, 31], [5, 16, 22, 31], [5, 16, 23, 31], [5, 16, 24, 31],
       [5, 16, 25, 31]]

## 카테고리 접근

In [5]:
category = driver.find_elements_by_css_selector('option')
for i in range(1):
    category[arr[i][0]].click()
    time.sleep(1)
    category[arr[i][1]].click()
    time.sleep(1)
    try:
        category[arr[i][2]].click()    
    except:
        print("왜이러지..")
    time.sleep(1)
    if arr[i][3] != 0:
        category[arr[i][3]].click()
        time.sleep(3)
    

## 정보 수집 과정

In [6]:
soup = bs(driver.page_source, 'html.parser')
posts = soup.select('div.summary.mato5.bold')[0].get_text()
subject_count = int(posts[11:])
subject_count = subject_count*16 + 5
# 과목당 16개 칼럼
for i in range(5, 21):
    try:
        contents = soup.select('td')[i].get_text()
        if (i-4)%16==7:
            contents = contents[1:3]
        if (i-4)%16==0:
            contents = contents[14:16]
        print(str(i)+", "+contents)
    except:
        print("finish")

5, 0
6, 교양과(서울)
7, 교양필수
8, 교양필수
9, 교양필수
10, 001009-1
11, 영어
12, 3
13, 12
14, 0/0/3/26
15, 2/2/2/8/12
16, 10
17, Andro N. Paris
18, 화56,금5
19, D0103-2,D0103-2
20, 불가


## parsing의 parameter 설정

In [13]:
temp_count = soup.select('td')[12].get_text()
temp_contents = soup.select('td')[18].get_text()

## 예제

In [4]:
print(parsing(1, "월6,화6"))
print(parsing(1, "월67"))
print(parsing(2, "화1,화2"))
print(parsing(2, "월67"))
print(parsing(2, "월910"))
print(parsing(2, "월567"))
print(parsing(2, "화1011"))
print(parsing(2, ""))
print(parsing(2, "화1,화2"))
print(parsing(4, "화45,금8,금9"))
print(parsing(4, "월6,목67,월7"))
print(parsing(4, "화7,수7,목78"))
print(parsing(4, "월8,금910,월9"))
print(parsing(4, "화89,금67"))
print(parsing(4, "월910,금89"))
print(parsing(4, "화67,금1011"))
print(parsing(4, "월1011,수1011"))
print(parsing(4, "월1234"))
print(parsing(6, "월123456"))
print(parsing(6, "화789,금789"))
print(parsing(6, "월6789,목6789"))

월6화6
월6월7
화1화2
월6월7
월9월10
월5월6월7
화10화11

화1화2
화4화5금8금9
월6목6목7월7
화7수7목7목8
월8금9금10월9
화8화9금6금7
월9월10금8금9
화6화7금10금11
월10월11수10수11
월1월2월3월4
월1월2월3월4월5월6
화7화8화9금7금8금9
월6월7월8월9목6목7목8목9


## parsing

In [1]:
def parsing(count, contents):
    length = len(contents)

    try:
        temp = contents[0]
        temp1 = contents[1]
        temp2 = contents[2]
        temp3 = contents[3]
        temp4 = contents[4]
        temp5 = contents[5]
        temp6 = contents[6]   
    except:
        print("", end="")
    
    temp_arr = contents.split(',')
    split_len = len(temp_arr)

    
    if count == 1:
        # 교시가 10의 자리인 경우 파싱하지 않음
        try:
            temp_arr[1] = temp_arr[1]
            return temp_arr[0]+temp_arr[1]  # ex. 월6,화7
        except:
            return temp+temp1+temp+temp2  # ex. 월67
    elif count == 2:
        try:
            temp_arr[1] = temp_arr[1]
            return temp_arr[0]+temp_arr[1]  # ex. 화1,화2
        except:
            if length == 2:
                return contents  # ex. 월1
            elif length == 3:
                return temp+temp1+temp+temp2  # ex. 월67
            elif length == 4:
                if contents[1:]=='910':
                    temp2 = contents[2:]
                    return temp+temp1+temp+temp2  # ex. 월910
                else:
                    return temp+temp1+temp+temp2+temp+temp3  # ex. 월567                
            elif length == 5:
                temp1 = contents[1:3]
                temp2 = contents[3:]
                return temp+temp1+temp+temp2  # ex. 화1011
            elif contents=="":
                return contents  # 비어있는 경우
    elif count == 3:
        # 같은경우 재끼는 def 필요?
        # 화10,수10,목10
            # 3개, 11개
        # 수9,수10,수11
            # 3개, 10개, 1 2 2
        # 월10,화10,목9
            # 3개, 10개, 2 2 1
        # 화56,목5,화56
            # 3개, 10개, 2 1 2 *  
        # 화2,수2,목2
            # 3개, 8개

        # 월1011,수1011
            # 2개, 11개
        # 화1011,목10
            # 2개, 9개, 22 2
        # 월910,수910
            # 2개, 9개, 12 12
        # 목234,목234
            # 2개, 9개, 111 111 *
        # 수8910,수10
            # 2개, 9개, 112 2 *
        # 화234,화34
            # 2개, 8개, 111 11 *
        # 월9,수1011
            # 2개, 8개, 1 22    
        # 월789,화7
            # 2개, 7개, 111 1
        # 화34,목34
            # 2개, 7개, 11 11
        # 화1,수910
            # 2개, 7개, 1 12
        # 목789,목9
            # 2개, 7개, 111 1 *
        # 금567,금7
            # 2개, 7개, 111 1 *
        # 화89,금10
            # 2개, 7개, 11 2
        # 월910,수9
            # 2개, 7개, 12 1      
        # 월12,목1
            # 2개, 6개, 11 1
        # 월7,목34
            # 2개, 6개, 1 11

        # 화111213
            # 1개, 7개, 222
        # 수678910
            # 1개, 7개, 11112       
        # 수91011
            # 1개, 6개, 122
        # 수12345
            # 1개, 6개, 11111
        # 월78910
            # 1개, 6개, 1112     
        # 목8910
            # 1개, 5개, 112
        # 월6789
            # 1개, 5개, 1111       
        # 목567
            # 1개, 4개
        # ""
            # 1개, 0개

        print(contents)
    elif count == 4:
        if split_len==3:
            a0 = temp_arr[0][0]
            a1 = temp_arr[0][1]                
            b0 = temp_arr[1][0]
            b1 = temp_arr[1][1]                
            c0 = temp_arr[2][0]
            c1 = temp_arr[2][1]
            if length==9:
                if is_num(contents[2]):
                    a2 = temp_arr[0][2]
                    return a0+a1+a0+a2+b0+b1+c0+c1  # 화45,금8,금9
                elif is_num(contents[4]) and is_num(contents[5]):
                    b2 = temp_arr[1][2]
                    return a0+a1+b0+b1+b0+b2+c0+c1  # 월6,목67,월7
                elif is_num(contents[7]):
                    c2 = temp_arr[2][2]
                    return a0+a1+b0+b1+c0+c1+c0+c2  # 화7,수7,목78
            elif length==10:
                b2 = temp_arr[1][2:]
                return a0+a1+b0+b1+b0+b2+c0+c1  # 월8,금910,월9
        elif split_len==2:
            a0 = temp_arr[0][0]
            a1 = temp_arr[0][1]                
            b0 = temp_arr[1][0]
            b1 = temp_arr[1][1]            
            if length==7:
                a2 = temp_arr[0][2]
                b2 = temp_arr[1][2]
                return a0+a1+a0+a2+b0+b1+b0+b2  # 화89,금67
            elif length==8:
                a2 = temp_arr[0][2:]
                b2 = temp_arr[1][2]
                return a0+a1+a0+a2+b0+b1+b0+b2  # 월910,금89
            elif length==9:
                a2 = temp_arr[0][2]
                b1 = temp_arr[1][1:3]
                b2 = temp_arr[1][3:]
                return a0+a1+a0+a2+b0+b1+b0+b2  # 화67,금1011
            elif length==11:
                a1 = temp_arr[0][1:3]
                a2 = temp_arr[0][3:]
                b1 = temp_arr[1][1:3]
                b2 = temp_arr[1][3:]
                return a0+a1+a0+a2+b0+b1+b0+b2  # 월1011,수1011
        elif split_len==1:
            return temp+temp1+temp+temp2+temp+temp3+temp+temp4  # 월1234
        
    elif count == 6:
        length = len(contents)
        if length == 7:
            return temp+temp1+temp+temp2+temp+temp3+temp+temp4+temp+temp5+temp+temp6  # 월456789
        elif length == 9:
            temp = temp_arr[0][0]
            temp1 = temp_arr[0][1]
            temp2 = temp_arr[0][2]
            temp3 = temp_arr[0][3]
            temp4 = temp_arr[1][0]
            temp5 = temp_arr[1][1]
            temp6 = temp_arr[1][2]
            temp7 = temp_arr[1][3]    
            return temp+temp1+temp+temp2+temp+temp3+temp4+temp5+temp4+temp6+temp4+temp7  # 화789,금789
        elif length == 11:
            temp = temp_arr[0][0]
            temp1 = temp_arr[0][1]
            temp2 = temp_arr[0][2]
            temp3 = temp_arr[0][3]
            temp4 = temp_arr[0][4]
            temp5 = temp_arr[1][0]
            temp6 = temp_arr[1][1]
            temp7 = temp_arr[1][2]               
            temp8 = temp_arr[1][3]               
            temp9 = temp_arr[1][4]               
            return temp+temp1+temp+temp2+temp+temp3+temp+temp4+temp5+temp6+temp5+temp7+temp5+temp8+temp5+temp9  # 월6789,목6789
    else:
        print("something error")


## is_num def

In [3]:
def is_num(num):
    if num=="1" or num=="2" or num=="3" or num=="4" or num=="5" or num=="6" or num=="7" or num=="8" or num=="9" or num=="0":
        return 1
    else :
        return 0

## 분석자료

4행 41열 배열 만들어서 41번 반복하면서 탐색, 크롤링
4 7 16
4 8 17
5 7 19
5 7 20 30
5 7 21 30
5 7 22 30
5 7 23 30
5 7 24 30
5 8 19 25
5 9 19 28
5 9 20 28
5 9 21 28
5 9 22 28
5 10 19 29
5 10 20 29
5 10 21 29
5 10 22 29
5 10 23 29
5 11 19
5 11 20 35
5 11 21 35
5 11 22 35
5 11 23 35
5 11 24 35
5 11 25 35
5 11 26 35
5 11 27 35
5 11 28 35
5 11 29 35
5 12 19 26
5 13 19
5 13 20 26
5 14 19 25
5 15 19 25
5 16 19 31
5 16 20 31
5 16 21 31
5 16 22 31
5 16 23 31
5 16 24 31
5 16 25 31

1: 서울
4: 교양
    7: 아빅교양
        16: 전체
    8: 일반교양
        17: 전체
5: 전공
    7: 공과대학
        19:전공공통
        20~24
            30: 전체학년
    8: 경영대학
        19: 경영학부
            25: 전체학년
    9: 문과대학
        19~22
            28: 전체학년
    10: 사범대학
        19~23
            29: 전체학년
    11: 미술대학
        19: 전공공통
        20~29
            35: 전체학년
    12: 건축대학
        19: 건축학부
            26: 전체학년
    13: 법과대학
        19: 전공공통
        20: 법학부
            26: 전체학년
    14: 경제학부
        19: 경제학부
            25: 전체학년
    15: 산업정보융합학부
        19: 디자인경영융합학부
            25: 전체학년
    16: 융합전공
        19~25
            31: 전체학년